# Turning Tuesdays

In financial markets, traders talk about the concept of "Turning Tuesday." The idea is that if a market is down for several days in a row, it will never turn around on a Friday because traders don't want to stay in a position over the weekend. Their pessism looms over the weekend. Likewise, on a Monday, traders have been sitting around depressed for the whole weekend (because the market is down) so Monday will stay down. It's on Tuesday that more rational heads prevail, so a down market will more likely "turn around" on a Tuesday.

### There is anectdotal evidence to support this....

In [1]:
from IPython.display import Image
Image(filename='/img/wsj_turnaround_tuesday.png') 

FileNotFoundError: [Errno 2] No such file or directory: '/img/wsj_turnaround_tuesday.png'

### What does the data say?.... Let's find out!

In [ ]:
import pandas as pd

### Get some data!

`#git clone https://github.com/talumbau/strata_data.git`

In [ ]:
sp500 = pd.read_csv("data/table.csv", index_col=0, parse_dates=True)

In [ ]:
sp500.head()

In [ ]:
sptt = sp500[["Adj Close"]]

In [ ]:
sptt.head()

In [ ]:
sptt.tail()

### If you have datetime data, it's almost always the right answer to use pandas


In [ ]:
sp500.index

### First question: which days did the market go up or down?

In [ ]:
def rel_change(x):
    return (x[0] - x[1])/x[1]

In [ ]:
sptt.assign?

In [ ]:
sptt = sptt.assign(UpDown=pd.rolling_apply(sptt, 2, rel_change))

In [ ]:
sptt.ix[0:20]

Hmm... we need to shift the results "up" one. January 11 was an "up" day, but its entry is one row too low in the "UpDown" column

In [ ]:
sptt.UpDown = sptt.UpDown.shift(-1)

In [ ]:
sptt.ix[0:20]

In [ ]:
sptt.head()

In [ ]:
sptt.head()

In [ ]:
sptt.tail()

### Next question: which day of the week corresponds to each row?

In [ ]:
sptt.index

In [ ]:
dir(sptt.index)

In [ ]:
sptt['day_num'] = sptt.index.dayofweek

In [ ]:
sptt.head(7)

### Groupby operations! Do calculations over every value that has the same day of week. Ex:

In [ ]:
sptt["Adj Close"].groupby(sptt.index.dayofweek).mean()

### Let's get more specific with the "Turning Tuesday Hypothesis":

We'll say a "down market" is a market where there are three or more days in a row where the market closes lower than it started. So, for example, for a Thursday, we are interested in what has happened on the three previous tradings days (typically the Monday, Tuesday, and Wednesday directly preceding the Thursday). With that as the definition of a "down market", the question is then, 'When we are in a down market, do we return to an up market more frequently on a Tuesday than on other days of the week?'

In [ ]:
def last_three_days_down(x):
    if x[1]<=0 and x[2]<=0 and x[3]<=0:
        return 1
    else:
        return 0

In [ ]:
sptt['ltdd'] = pd.rolling_apply(sptt["UpDown"], 4, last_three_days_down).shift(-3)

In [ ]:
sptt.head()

In [ ]:
len(sptt)

In [ ]:
import numpy as np

In [ ]:
sptt.head()

In [ ]:
sptt['turned'] = (sptt['ltdd'] > 0.) & (sptt['UpDown'] > 0.)

In [ ]:
sptt.head(50)

In [ ]:
sp_turning_days_totals = sptt.turned.groupby(sptt.index.dayofweek).sum()

In [ ]:
%matplotlib inline

In [ ]:
sp_turning_days_totals.plot()

In [ ]:
sp_turning_days_totals